In [1]:
from random import randint
import tensorflow as tf
import tensorflow.contrib.slim as slim
from sklearn.utils import shuffle

In [2]:
import cifar10

In [3]:
cifar10.data_path = "data"

In [4]:
cifar10.maybe_download_and_extract()

Data has apparently already been downloaded and unpacked.


In [5]:
class_names = cifar10.load_class_names()

Loading data: data/cifar-10-batches-py/batches.meta


In [6]:
class_names

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [7]:
images_train, cls_train, labels_train = cifar10.load_training_data()

Loading data: data/cifar-10-batches-py/data_batch_1
Loading data: data/cifar-10-batches-py/data_batch_2
Loading data: data/cifar-10-batches-py/data_batch_3
Loading data: data/cifar-10-batches-py/data_batch_4
Loading data: data/cifar-10-batches-py/data_batch_5


In [8]:
images_train.shape

(50000, 32, 32, 3)

In [9]:
cls_train.shape

(50000,)

In [10]:
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_integer("RE_IMG_SIZE", 24, "resized image size")
#tf.app.flags.DEFINE_string()
#tf.app.flags..DEFINE_integer("image_width", 61, "image width")

In [11]:
def distort_images(image, training):
    if training:
        # Randomly crop the input image.
        distorted_image = tf.random_crop(image, size=[FLAGS.RE_IMG_SIZE, FLAGS.RE_IMG_SIZE, 3])
        distorted_image = tf.image.random_brightness(distorted_image, max_delta=63)
        
        with tf.name_scope('distort_color'):
            color_ordering = randint(0,1) # 0, 1 random number 생성

            if color_ordering == 0:
                #print('0')
                distorted_image = tf.image.random_brightness(distorted_image, max_delta=32/255)
                distorted_image = tf.image.random_saturation(distorted_image, lower=0.5, upper=1.5)
                distorted_image = tf.image.random_hue(distorted_image, max_delta=0.05)
                distorted_image = tf.image.random_contrast(distorted_image, lower=0.5, upper=1.5)
            else:
                #print('1')
                distorted_image = tf.image.random_brightness(distorted_image, max_delta=32/255)
                distorted_image = tf.image.random_contrast(distorted_image, lower=0.5, upper=1.5)
                distorted_image = tf.image.random_saturation(distorted_image, lower=0.5, upper=1.5)
                distorted_image = tf.image.random_hue(distorted_image, max_delta=0.05)
            
            distorted_image = tf.clip_by_value(distorted_image, 0.0, 1.0)
            
    else:
        distorted_image = tf.image.resize_image_with_crop_or_pad(image,
                                                              target_height = FLAGS.RE_IMG_SIZE,
                                                              target_width = FLAGS.RE_IMG_SIZE)
        
        distorted_image = tf.clip_by_value(distorted_image, 0.0, 1.0)
        distorted_image = tf.cast(distorted_image, tf.float32)
    
    return distorted_image

In [12]:
def pre_process(images, training):
    images = tf.map_fn(lambda image: distort_images(image, training), images)
    
    return images

In [13]:
def inference(x, y_, training):
    
    # Convolution Layer1
    with tf.variable_scope('conv_block1'):
        conv1 = tf.contrib.layers.convolution2d(inputs=x, num_outputs=64,
                                        kernel_size=5,
                                        stride=1,
                                        padding='SAME',
                                        activation_fn=None,
                                        weights_initializer=tf.truncated_normal_initializer(stddev=0.05),
                                        biases_initializer=None,
                                        scope='conv1'
                                           )

        pool_1 = tf.contrib.layers.max_pool2d(inputs=conv1,
                                              kernel_size=3,
                                              stride=2,
                                              padding='SAME',
                                              scope='pool_1')

        batch_1 = tf.contrib.layers.batch_norm(pool_1, 
                                               center=True,
                                               scale=True,
                                               is_training=training,
                                               scope='bn')
        
        act1 = tf.nn.relu(batch_1, name='activation_1')
    
    # Convolution Layer2
    with tf.variable_scope('conv_block2'):
        conv2 = tf.contrib.layers.convolution2d(inputs=act1, num_outputs=64,
                                        kernel_size=5,
                                        stride=1,
                                        padding='SAME',
                                        activation_fn=None,
                                        weights_initializer=tf.truncated_normal_initializer(stddev=0.05),
                                        biases_initializer=None
                                        )

        pool_2 = tf.contrib.layers.max_pool2d(inputs=conv2,
                                              kernel_size=3,
                                              stride=2,
                                              padding='SAME')

        batch_2 = tf.contrib.layers.batch_norm(pool_2,
                                               center=True,
                                               scale=True,
                                               is_training=training,
                                               scope='bn')
        
        act2 = tf.nn.relu(batch_1, name='activation_2')
    
    
    # Fully Connected Layer
    flatten = tf.contrib.layers.flatten(act2, scope='flatten')
    
    with tf.variable_scope('fc_layer'):
        fc1 = tf.contrib.layers.fully_connected(flatten,num_outputs=384,
                                            activation_fn=tf.nn.relu,
                                            weights_initializer=tf.truncated_normal_initializer(stddev=0.05),
                                            biases_initializer=tf.truncated_normal_initializer(stddev=0.05))

        fc2 = tf.contrib.layers.fully_connected(fc1,num_outputs=192,
                                            activation_fn=tf.nn.relu,
                                            weights_initializer=tf.truncated_normal_initializer(stddev=0.05),
                                            biases_initializer=tf.truncated_normal_initializer(stddev=0.05))
    
    # Logits(final Layer)
    with tf.variable_scope('Logits'):
        logits = tf.contrib.layers.fully_connected(fc2,num_outputs=10,
                                            activation_fn=None,
                                            weights_initializer=tf.truncated_normal_initializer(stddev=0.05),
                                            biases_initializer=tf.truncated_normal_initializer(stddev=0.05))
    
    
    
    one_hot = tf.one_hot(y_, 10, name='one_hot')
    
    with tf.name_scope('loss'):
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, one_hot)
        loss = tf.reduce_mean(cross_entropy)
        
    
    # Evaluate Pipeline
    correct_prediction = tf.equal(tf.arg_max(logits, 1), tf.argmax(one_hot, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
    
    return logits, loss, accuracy
    

In [15]:
def train(images, labels, epochs, batch_size):
    
    train_size = len(images) #50000
    
    x = tf.placeholder(tf.float32, [None, FLAGS.RE_IMG_SIZE, FLAGS.RE_IMG_SIZE, 3], name='x')
    y = tf.placeholder(tf.int32, [None], name='y')
    
    logits, loss, accuracy = inference(x, y, True)
    tf.summary.scalar('loss', loss)
    tf.summary.scalar('accuracy', accuracy)
    
    
    with tf.name_scope('ADAM'):
            batch_step = tf.Variable(initial_value=0,
                                      name='global_step',
                                      trainable=False)
            
            learning_rate = tf.train.exponential_decay(1e-4,
                                                       global_step= batch_step* batch_size,
                                                       decay_steps= train_size,
                                                       decay_rate=0.95,
                                                       staircase=True
                                                      )
            train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=batch_step)
    
    
    # Keep track of learning rate
    tf.summary.scalar('learning_rate', learning_rate)
    
    init_op = tf.global_variables_initializer()
    
    # Build the summary Tensor based on the TF collection of Summaries.
    summary = tf.summary.merge_all()
    
    with tf.Session() as sess:
        sess.run(init_op)
        
        # Instantiate a SummaryWriter to output summaries and the Graph.
        summary_writer = tf.summary.FileWriter('./log', sess.graph)
        
        batch_step = 0
        for e in range(epochs):
            
            # Shake input datas
            x_train, y_train = shuffle(images, labels)
            
            for offset in range(0, train_size, batch_size):
                batch_step +=1
                print('batch_step', batch_step)
                end = offset + batch_size
                x_batch, y_batch = x_train[offset:end], y_train[offset:end]

                # distort images, randomly
                x_batch = pre_process(x_batch, training=True)
                x_batch = sess.run(x_batch)
                
                feed_train = {x: x_batch, y: y_batch}

                # parameter update(training)
                sess.run(train_op, feed_dict = feed_train)
                #train_loss = sess.run(loss, feed_dict = feed_train)
                train_accuracy = sess.run(accuracy, feed_dict = feed_train)
                
                
                # print()
                if(batch_step % 10 == 0):
                    print('Epoch:{0}, batch_step:{1}, train_accuracy,{2}'.format(e, batch_step, train_accuracy))
                
                # Update the events file.
                summary_str = sess.run(summary, feed_dict=feed_train)
                summary_writer.add_summary(summary_str, batch_step)
                summary_writer.flush()
        
        
        try:
            saver
        except NameError:
            saver = tf.train.Saver()
        saver.save(sess, 'cifar10')
        print("Model saved")
            
            
            
    
    

In [16]:
tf.reset_default_graph()
train(images=images_train, labels=cls_train, epochs=1, batch_size=128)

batch_step 1
batch_step 2
batch_step 3
batch_step 4
batch_step 5
batch_step 6
batch_step 7
batch_step 8
batch_step 9
batch_step 10
Epoch:0, batch_step:10, train_accuracy,0.1171875
batch_step 11
batch_step 12
batch_step 13
batch_step 14
batch_step 15
batch_step 16
batch_step 17
batch_step 18
batch_step 19
batch_step 20
Epoch:0, batch_step:20, train_accuracy,0.09375
batch_step 21
batch_step 22
batch_step 23
batch_step 24
batch_step 25
batch_step 26
batch_step 27
batch_step 28
batch_step 29
batch_step 30
Epoch:0, batch_step:30, train_accuracy,0.0546875
batch_step 31
batch_step 32
batch_step 33
batch_step 34
batch_step 35
batch_step 36
batch_step 37
batch_step 38
batch_step 39
batch_step 40
Epoch:0, batch_step:40, train_accuracy,0.078125
batch_step 41
batch_step 42
batch_step 43
batch_step 44
batch_step 45
batch_step 46
batch_step 47
batch_step 48
batch_step 49
batch_step 50
Epoch:0, batch_step:50, train_accuracy,0.1328125
batch_step 51
batch_step 52
batch_step 53
batch_step 54
batch_step 

In [22]:
tf.reset_default_graph()

In [18]:
images_test, cls_test, labels_test = cifar10.load_test_data()

Loading data: data/cifar-10-batches-py/test_batch


In [23]:
with tf.Session() as sess:
    # x, y, logits, train, accuracy = build_graph(model_config)
    print("check1")
    x = tf.placeholder(tf.float32, [None, FLAGS.RE_IMG_SIZE, FLAGS.RE_IMG_SIZE, 3], name='x')
    y = tf.placeholder(tf.int32, [None], name='y')
    
    logits, loss, accuracy = inference(x, y, False)
    print("check2")
    loader = tf.train.import_meta_graph('cifar10.meta')
    loader.restore(sess, tf.train.latest_checkpoint('./'))
    print("check3")
    batch_size = 128
    valid_size = len(images_test)
    
    # to get session using in training.
    sess = tf.get_default_session()
    total_accuracy = 0
    print("check4")
    for offset in range(0, valid_size, batch_size):
        batch_x, batch_y = images_test[offset:offset+batch_size], cls_test[offset:offset+batch_size]
        print("check5")
        # distort images, randomly
        batch_x = pre_process(batch_x, training=False)
        batch_x = sess.run(batch_x)
        print("check6")
        accuracy_value = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
        print(">>>>>>>>>>>>>>", accuracy_value)
        total_accuracy += (accuracy_value * batch_size)
    
    test_accuracy = total_accuracy / valid_size
    
    print("Test Accuracy = {:.3f}".format(test_accuracy))

check1
check2
check3
check4
check5


InvalidArgumentError: TensorArray dtype is double but Op is trying to write dtype float.
	 [[Node: map/while/TensorArrayWriteV2_1 = TensorArrayWriteV2[T=DT_FLOAT, _class=["loc:@map/TensorArray_3"], _device="/job:localhost/replica:0/task:0/cpu:0"](map/while/TensorArrayWriteV2_1/Enter, map/while/Identity_2, map/while/Cast, map/while/Identity_3)]]

Caused by op 'map/while/TensorArrayWriteV2_1', defined at:
  File "/Users/jihobak/.pyenv/versions/miniconda3-4.0.5/envs/CarND-LeNet-Lab/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/jihobak/.pyenv/versions/miniconda3-4.0.5/envs/CarND-LeNet-Lab/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/jihobak/.pyenv/versions/miniconda3-4.0.5/envs/CarND-LeNet-Lab/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/jihobak/.pyenv/versions/miniconda3-4.0.5/envs/CarND-LeNet-Lab/lib/python3.5/site-packa